<a href="https://colab.research.google.com/github/mirignak/py_depression_eldery/blob/main/research_method_imms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## data: 실태조사 원본데이터
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/고연방_분석/2020_TheOld_suvery_721(UTF-8).csv', low_memory=False)



In [ ]:
#### 기본 통계분석
# 연령, 성별(1:남성, 2:여성),주관적 건강상태(1-5)
df = data[['개인 ID', '노인 조사 대상자 만연령', '노인 조사 대상자 성별', '평소의 건강상태']].copy()
df.columns = ['id', 'age', 'gender', 'health']

# 혼인상태(0:유배우자, 1: 무배우자)
df['marital_stat'] = data['노인 조사 대상자 혼인상태'].apply(lambda x: 0 if x == 2 else 1)
# 교육수준(0:중학교 이하, 1:고등학교 이상)
df['edu_level'] = data['노인 조사 대상자 교육수준'].apply(lambda x: 0 if x in [0,1,2,3,4] else 1)
# 경제활동 여부(1: 은퇴, 0: 일하는중, 일한적 없음)
df['econo_stat'] = data['현재 경제활동 여부'].apply(lambda x: 1 if x==2 else 0)
# 가구소득
df['income'] = np.round(data['가구_총수입액_금액'] / np.sqrt(data['총 가구원 수']))
max_income = df['income'].max()
min_income = df['income'].min()

#### 종속변수
#우울
depression_columns = ['우울증척도_현재생활 만족', '우울증척도_의욕저하', '우울증척도_헛된 느낌', '우울증척도_지루함', '우울증척도_상쾌함',
                      '우울증척도_불안함', '우울증척도_마음의 즐거움', '우울증척도_절망적 느낌', '우울증척도_나가기 싫음', '우울증척도_기억력 나쁨(동년배 비교)',
                      '우울증척도_살아있음에 대한 즐거움', '우울증척도_쓸모없음', '우울증척도_기력좋음', '우울증척도_희망없음', '우울증척도_다른사람들보다 처지가 더 못함'
                      ]
df['depression'] = data[depression_columns].sum(axis=1)

#### 매개변수
# 사회접촉(연락빈도. 0 ~ 32)
contact_columns = ['비동거 생존자녀 전체_왕래 빈도(지난 1년 간)', '비동거 생존자녀 전체_연락 빈도(지난 1년 간)', '비동거 손자녀 전체_왕래 빈도(지난 1년 간)', '비동거 손자녀 전체_연락 빈도(지난 1년 간)'
  , '형제/자매를 포함한 친/인척과의 왕래 빈도(지난 1년 간)', '형제/자매를 포함한 친/인척과의 연락 빈도(지난 1년 간)', '친구/이웃/지인과의 왕래 빈도(지난 1년 간)', '친구/이웃/지인과의 연락 빈도(지난 1년 간)']

def calc_social_contact(row):
    total = sum([4 if val >= 1 and val <= 3 else 3 if val == 4 else 2 if val == 5 else 1 if val == 6 else 0 if val == 7 else 0 for val in row.values])
    return total

df['social_contact'] = data[contact_columns].apply(calc_social_contact, axis=1)

####조절변수
# 디지털기기 활용(1:예, 0:아니오 및 해당없음)
use_digital_columns = ['전자기기 활동여부_메시지 받기(문자, 카카오톡, 텔레그램 등)', '전자기기 활동여부_메시지 보내기(문자, 카카오톡, 텔레그램 등)', '전자기기 활동여부_정보 검색 및 조회(뉴스, 날씨 등)',
                       '전자기기 활동여부_사진 또는 동영상 촬영', '전자기기 활동여부_음악 듣기(MP3, 라디오 등)', '전자기기 활동여부_게임',
                       '전자기기 활동여부_동영상 보기(영화, TV프로그램, 유튜브 등)', '전자기기 활동여부_소셜네트워크 서비스(블로그, 커뮤니티, 밴드, 트위터, 페이스북, 인스타그램 등)',
                       '전자기기 활동여부_전자상거래(온라인 쇼핑, 예매, 예약 등)', '전자기기 활동여부_금융거래(인터넷 뱅킹, 증권 등)', '전자기기 활동여부_애플리케이션 검색 및 설치']
df['use_digital'] = data[use_digital_columns].apply(lambda x: sum([1 if val == 1 else 0 for val in x]), axis=1)

# 생활정보 이용 어려움(1:어려움이 있다, 0:없다 및 해당없음)
## 이 데이터 쓰는게 맞을까???
diff_info_access_columns = ['생활 필요 정보 이용시 어려움_정부나 공공기관(건강보험공단 등)의 우편물 또는 공지 사항 등에 대해 용어 또는 내용을 이해하기 어려움',
                          '생활 필요 정보 이용시 어려움_공공기관이나 마을(아파트), 마트 등에서 정보를 제공하는 공지사항의 글씨 크기가 작아서 보기 어려움',
                          '생활 필요 정보 이용시 어려움_각종 정보 제공이나 서비스 신청이 온라인/인터넷을 중심으로 이루어져서 이용하기 어려움']
df['diff_info_access'] = data[use_digital_columns].apply(lambda x: sum([1 if val == 1 else 0 for val in x]), axis=1)

# 정보화기기 이용 불편함(1:어려움이 있다, 2:없다, 9: 해당없음)
## TODO : 데이터를 단순화할지, 그대로 사용할지 결정 필요
discomfort_it_use_columns = ['정보화기기 이용 시 불편함_기차/고속버스/시외 버스 예매(온라인 예매 중심으로 자리가 없어서)', '정보화기기 이용 시 불편함_식당 등에서 기계(키오스크) 주문',
                  '정보화기기 이용 시 불편함_은행에서의 ATM 사용 또는 은행 점포 감소', '정보화기기 이용 시 불편함_신용카드, 체크카드 등 카드만 이용 가능한 점포 증가']

#print(df.head(20))



In [ ]:
## 추가 변수 후보
# 삶에 대한 만족도

# 노인의 사회존중(1-5, 높아질수록 존중 못받음)
df['respect'] = data['노인의 사회적 존중 정도'].copy()

# 노인이라서 무시당한 경험(0-7, 높아질수록 경험많음)
ignore_experience_columns = ['지난 1년 간 노인이기 때문에 무시(차별) 당했다고 느낀 경험_대중교통 이용 시', '지난 1년 간 노인이기 때문에 무시(차별) 당했다고 느낀 경험_식당, 커피숍 이용 시',
                             '지난 1년 간 노인이기 때문에 무시(차별) 당했다고 느낀 경험_대형마트 및 백화점 등 판매 시설 이용 시',
                             '지난 1년 간 노인이기 때문에 무시(차별) 당했다고 느낀 경험_주민센터, 구청 등 공공기관 이용 시',
                             '지난 1년 간 노인이기 때문에 무시(차별) 당했다고 느낀 경험_병원 등 의료시설 이용 시',
                             '지난 1년 간 노인이기 때문에 무시(차별) 당했다고 느낀 경험_일터(직장)',
                             '지난 1년 간 노인이기 때문에 무시(차별) 당했다고 느낀 경험_가족 내 주요 의사 결정시']
df['ignore_exp'] = data[ignore_experience_columns].apply(lambda x: sum([1 if val == 1 else 0 for val in x]), axis=1)

pd.set_option('display.max_columns', 30)  # 출력 최대 컬럼
pd.set_option('display.width', 1000) # 출력 너비
print(df.head(20))



         id  age  gender  health  marital_stat  edu_level  econo_stat  income  use_digital_cam  use_digital_music  use_digital_game  use_digital_movie  use_digital_sns  use_digital_shop  use_digital_bank  use_digital_app  depression  use_digital  info_access_word  info_access_fontsize  info_access_online  social_contact  respect  ignore_exp
0   1000001   74       2       2             1          0           3   745.0                2                  2                 2                  2                2                 2                 2                2          18            3                 1                     1                   1              30        3           5
1   1000101   68       1       3             0          1           2  3889.0                2                  2                 2                  1                2                 2                 2                2          25            3                 2                     2                   1         

# 새 섹션

# 새 섹션

## 종속변수: 우울
 - SGDS-K(단축형 노인우울척도, Short Form of Geriatric Depression Scale)의 15개 항목으로 측정
 - 각 항목의 점수를 합산(0~15)
 - 5~9점: 우울의심, 10점 이상: 우울증
 - 척도의 신뢰도를 확인하기 위해 Cronbach a값 측정

## 독립변수: 은퇴
 - 은퇴 : "현재 모든 경제활동을 하지 않는다"
 - 비은퇴 : 은퇴가 아닌 모든 상태
 - 참고) 1: 일하는중, 2: 은퇴, 3: 일한적없음

## 매개변수: 사회접촉
 - 관련 컬럼:
    - 비동거 생존자녀 전체_왕래 빈도(지난 1년 간)
    - 비동거 생존자녀 전체_연락 빈도(지난 1년 간)
    - 비동거 손자녀 전체_왕래 빈도(지난 1년 간)
    - 비동거 손자녀 전체_연락 빈도(지난 1년 간)
    - 형제/자매를 포함한 친/인척과의 왕래 빈도(지난 1년 간)
    - 형제/자매를 포함한 친/인척과의 연락 빈도(지난 1년 간)
    - 친구/이웃/지인과의 왕래 빈도(지난 1년 간)
    - 친구/이웃/지인과의 연락 빈도(지난 1년 간)


 - 사회접촉은 Segel-Karpas와 Lachman(2016)이 제시한 사회접
촉 개념을 기반으로 재구성하였다. 즉, 지난 1년 동안 비동거 자녀, 손자녀, 친인척을
포함한 가족 집단과 친구 및 지인을 포함한 사회집단과의 연락 및 왕래의 빈도를 합산하였다. 각 문항의 구체적인 값은 “⓪왕래(연락)를 거의 하지 않는다, ① 1년에 1-2회 정도, ② 3개월에 1-2회 정도, ③ 한 달에 1-2회 정도, ④ 일주일에 1회 이상”으로 코딩한 후 합산하였다. 즉, 점수범위는 0점에서 32점으로 점수가 높을수록 사회접촉을 활발하게 하는 것으로 해석하였다.

## 조절변수: 전자기기 활용능력
- 전자기기 활동여부_메시지 받기(문자, 카카오톡, 텔레그램 등)|
전자기기 활동여부_메시지 보내기(문자, 카카오톡, 텔레그램 등)|
전자기기 활동여부_정보 검색 및 조회(뉴스, 날씨 등)|
전자기기 활동여부_사진 또는 동영상 촬영|
전자기기 활동여부_음악 듣기(MP3, 라디오 등)|
전자기기 활동여부_게임|
전자기기 활동여부_동영상 보기(영화, TV프로그램, 유튜브 등)|
전자기기 활동여부_소셜네트워크 서비스(블로그, 커뮤니티, 밴드, 트위터, 페이스북, 인스타그램 등)|
전자기기 활동여부_전자상거래(온라인 쇼핑, 예매, 예약 등)|
전자기기 활동여부_금융거래(인터넷 뱅킹, 증권 등)|
전자기기 활동여부_애플리케이션 검색 및 설치|

## 통제변수
- 연령
- 성별 (1:남성, 2:여성)
- 혼인상태 (0:유배우자, 1: 무배우자)
- 교육수준 (0:중학교 이하, 1:고등학교 이상)
- 가구소득 (max: 85025.0 , min: 46.0 만원)
    - OECD 균등화 가구소득 기준. (가구소득)/(가구원수의 제곱근)
- 주관적 건강



추가 조사가 필요한 항목: 키오스크 확산, 시간이 흘렀음,

### 노인의 키오스크 이용 경험에 대해
- 키오스크 서비스 실패: 시니어들의 부정적인 키오스크 이용 경험을 중심으로(김효정, 2021)
   - 키오스크 서비스 실패를 경험한 50-70대 시니어 소비자의 반응
   - '노력형’, ‘답보형’, ‘표출형’이라는 세 가지 유형의 소비자 집단이 도출되었다
     - 노력형: 키오스크 이용 역량을 향상시키기 위해 노력하는 집단
     - 답보형: 키오스크 이용으로 발생하는 전환비용을 심적으로 계산하는 집단
     - 표출형: 키오스크 서비스 실패로 발생된 부정적 감정을 타인에게 표출하는 집단

  - 한계: 3개월 내 키오스크 이용 경험이 있는 50대 이상을 대상으로 함(경험이 없는 시니어는 포함되지 않음)
- 2021년 디지털정보격차 실태조사(한국지능정보사회진흥원, 2021)에 따르면 노령층의 디지털정보화 종합수준은 69.1%로 같은 정보 취약계층으로 분류되는 저소득층(95.4%), 결혼이민자(89.5%), 북한 이탈 주민(89.4%), 장애인(81.7%) 등과 비교해 보았을 때 가장 낮은 수준
- **노령층의 키오스크 사용을 통해서 본 사회의 차별과 배제
: “저걸 통과해야 먹을 수 있다”(류수민, 2022)**
  - 키오스크 활용 능력이 떨어지는 노령층을 ‘비정상’인 범주에 속하는 대상으로 보게 된다면 이는 이들을 낙인찍어 사회에서 타자화하는 효과를 가져오게 된다
  - 노령층으로 하여금 단순히 시대에 뒤떨어진다는 감정적인 고통을 느끼게 하는 것뿐만 아니라 공간에서 환영받지 못하며, 먹기 위해서는 저것을 통과해야 한다는 절박함을 느끼게 한다


### 고령자를 위한 키오스크 사용 개선에 관한 연구가 활발하게 진행되고 있음
  - **고령자의 심리적 요인을 고려한 키오스크 사용경험 개선 제안 (정민경, 2022)**
    : 사용자 인터페이스 개선에 치중한 기존 연구와 방향성을 달리하여 고령자의 기술수용심리에 초점을 맞춘 키오스크 사용자경험 개선 방안을 제시

- **고령자의 디지털 소외 완화를 위한 디자인 제안 : 키오스크 인터페이스를 중심으로(김민주, 2020)**
- **병원 키오스크 UI 디자인에 대한 시니어 세대의 사용성 평가 연구 -서울 H 대학병원 키오스크 UI 디자인을 중심으로-(전서영, 2020)** : 시니어의 키오스크 사용 어려움과 관련하여 시니어의 접근성이 많은 병원의 무인시스템 키오스크 UI디자인에 대한 평가 및 개선방향 제시

